In [1]:
import pandas as pd
import simplejson
import numpy as np
import pprint

In [8]:
def mr_proc_results(raw_results):
  

  # Removing the f1 score for a bit
  pr_results = [[[acc_score, f1_score, round(sum(qf_s)/11,2),round(np.std(qf_s),2)],
  #pr_results = [[[acc_score, round(sum(qf_s)/11,2),round(np.std(qf_s),2)],
                 [acc_score_wp1, f1_score_wp1, round(sum(qf_s_wp1)/11,2),round(np.std(qf_s_wp1),2)]] 
                for ([acc_score, qa_s, aa_s], [f1_score, qf_s, af_s],
                     [acc_score_wp1, qa_s_wp1, aa_s_wp1], [f1_score_wp1, qf_s_wp1, af_s_wp1]) 
                in raw_results]

  # Throw the list in an np array
  pr_arr = np.array(pr_results)



  return(np.mean(pr_arr,axis=0).flatten())

In [3]:
### Config

# Path to data 
folder_path = "Results/Original/"

# List of results to process
all_res = ["tr","tr_qa","tr_set","bl","bl_qa","bl_set"]
# Filename prefix for each result type
res_fpath = ["tr_a_","tr_qa_","tr_qa_","bl_a_","bl_qa_","bl_qa_"]

# Base filenames = names of all augmentation strategies
base_files = ["orig","plus","dict","phrase","reord","fasttext","glove","ppdb","wordnet"]
# Set filenames = names of all combinations of data
set_files = ["ab_lq","ab_hq","all_lq","all_hq"]

In [4]:
# Initialize the different groups of results
results = {}

for res_type in all_res:
    results[res_type] = {}

In [5]:
# Load the data
for r_type,r_path in zip(all_res,res_fpath):
    # Base results
    if r_type not in ["tr_set","bl_set"]:
        # Load  all relevant results, go through all possible augmentations
        for cur_aug in base_files:
            # Results for basic augmentation : FOLDER/prefix_AUGNAME.txt    
            cur_fname = folder_path + r_path + cur_aug + ".txt"
            # Open and load the file
            with open(cur_fname,"r") as inp:
                results[r_type][cur_aug] = simplejson.load(inp)
    # Set results
    else:
        # Load  all relevant results, go through all possible sets
        for cur_aug in set_files:
            # Results for set augmentation : FOLDER/prefix_SETNAME.txt    
            cur_fname = folder_path + r_path + cur_aug + ".txt"
            # Open and load the file
            with open(cur_fname,"r") as inp:
                results[r_type][cur_aug] = simplejson.load(inp)        

In [6]:
# Initialize the processed results dictionary
proc_res_df = {}

# Loop through all different models in results, including 'combined' if it exists
for r_type in results:
    if r_type == 'combined' and isinstance(results['combined'], list):
        # Process 'combined' as a list directly without keys
        proc_res_df[r_type] = pd.DataFrame(data=[mr_proc_results(result) for result in results['combined']])
        
        # Assign a unique index to each row in the combined DataFrame
        proc_res_df[r_type].index = [f"{r_type}_result_{i+1}" for i in range(len(results['combined']))]
    else:
        # Process other entries in 'results' as dictionaries with nested keys
        proc_res_df[r_type] = pd.DataFrame(data=[mr_proc_results(results[r_type][aug]) for aug in results[r_type].keys()])
        
        # Fix the index to include model type and augmentation name
        proc_res_df[r_type].index = [r_type + "_" + aug for aug in results[r_type].keys()]

    # Set consistent column names for all DataFrames
    proc_res_df[r_type].columns = ['Acc', "F1", "F1-Q", "F1-STD", "UK20-Acc", "UK20-F1", "UK20-F1-Q", "UK20-F1-STD"]





In [7]:
pd.set_option('expand_frame_repr', False)
pp = pprint.PrettyPrinter(indent=4)
for r_type in proc_res_df:
    pp.pprint(proc_res_df[r_type])
    print()

               Acc     F1   F1-Q  F1-STD  UK20-Acc  UK20-F1  UK20-F1-Q  UK20-F1-STD
tr_orig      0.898  0.894  0.845   0.052     0.854    0.850      0.795        0.029
tr_plus      0.898  0.898  0.863   0.045     0.857    0.853      0.802        0.030
tr_dict      0.926  0.924  0.902   0.033     0.858    0.852      0.801        0.028
tr_phrase    0.923  0.923  0.904   0.033     0.857    0.853      0.803        0.028
tr_reord     0.925  0.925  0.905   0.033     0.857    0.855      0.805        0.029
tr_fasttext  0.919  0.918  0.893   0.036     0.854    0.850      0.800        0.030
tr_glove     0.917  0.917  0.892   0.036     0.853    0.850      0.799        0.027
tr_ppdb      0.921  0.920  0.897   0.035     0.856    0.853      0.803        0.030
tr_wordnet   0.919  0.920  0.896   0.035     0.853    0.851      0.800        0.029

                  Acc     F1   F1-Q  F1-STD  UK20-Acc  UK20-F1  UK20-F1-Q  UK20-F1-STD
tr_qa_orig      0.925  0.925  0.877   0.059     0.891    0.889      0.83